In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import matplotlib                  # 2D Plotting Library
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns              # Python Data Visualization Library based on matplotlib
import geopandas as gpd            # Python Geospatial Data Library
plt.style.use('fivethirtyeight')
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.figure_factory as ff
from plotly import subplots
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)


from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

from wordcloud import WordCloud
from collections import Counter


import folium
import folium.plugins

import matplotlib.dates as mdates
import plotly.express as px
from datetime import date, timedelta
from sklearn.cluster import KMeans
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import plotly.offline as py
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from keras.models import Sequential
from keras.layers import LSTM,Dense
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

from fbprophet.plot import plot_plotly
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
import plotly.offline as py
init_notebook_mode(connected=True)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

![corona](https://www.inquirer.com/resizer/CiwgX_mknaEMHprl0soM8TCazo0=/1400x932/smart/arc-anglerfish-arc2-prod-pmn.s3.amazonaws.com/public/PCDN4SDFNVHJRI4VRFRA43UPH4.jpg)

Coronaviruses (CoV) are a large family of viruses that cause illness ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS-CoV) and Severe Acute Respiratory Syndrome (SARS-CoV). A novel coronavirus (nCoV) is a new strain that has not been previously identified in humans.  

Coronaviruses are zoonotic, meaning they are transmitted between animals and people.  Detailed investigations found that SARS-CoV was transmitted from civet cats to humans and MERS-CoV from dromedary camels to humans. Several known coronaviruses are circulating in animals that have not yet infected humans. 

Common signs of infection include respiratory symptoms, fever, cough, shortness of breath and breathing difficulties. In more severe cases, infection can cause pneumonia, severe acute respiratory syndrome, kidney failure and even death. 

Standard recommendations to prevent infection spread include regular hand washing, covering mouth and nose when coughing and sneezing, thoroughly cooking meat and eggs. Avoid close contact with anyone showing symptoms of respiratory illness such as coughing and sneezing.

![oronavirus](https://www.aljazeera.com/mritems/Images/2020/1/24/e26d3593e9844d4f82622b0dfcfb6db1_18.jpg)

In [ ]:
# df1 = pd.read_csv("/kaggle/input/novel-corona-virus-2019-dataset/2019_nCoV_data.csv")
df = pd.read_csv("/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv")


conf_df = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv')


In [ ]:
df.head().style.background_gradient(cmap='RdGy')

In [ ]:
conf_df.head().style.background_gradient(cmap='RdGy')

In [ ]:
conf_df = conf_df.rename(columns={"Province/State": "province", "Country/Region": "country"})

In [ ]:
df = df.rename(columns={"Province/State": "province", "Country/Region": "country","Last Update":"Last_Update"})

In [ ]:
print(df.columns.values)

In [ ]:
df.info()

In [ ]:
conf_df.info()

In [ ]:
del df['SNo']

In [ ]:
df.describe().style.background_gradient(cmap='BrBG')

In [ ]:
df.describe(include=['O'])

Finding the correlation among columns

In [ ]:
fig=plt.gcf()
fig.set_size_inches(10,7)
fig=sns.heatmap(df.corr(),annot=True,cmap='cubehelix',linewidths=1,linecolor='k',square=True,mask=False, vmin=-1, vmax=1,cbar_kws={"orientation": "vertical"},cbar=True)


In [ ]:
df.describe().plot(kind = "area",fontsize=27, figsize = (20,8), table = True,colormap="rainbow")
plt.xlabel('Statistics',)
plt.ylabel('Value')
plt.title("General Statistics of Corona Virus Dataset")

In [ ]:
fig=sns.jointplot(x='Confirmed',y='Recovered',data=df,kind='reg')

In [ ]:
import seaborn as sns; sns.set(style="ticks", color_codes=True)

g = sns.pairplot(df)

In [ ]:
# df['Date'] = pd.to_datetime(df['Date'])
df['Last_Update'] = pd.to_datetime(df['Last_Update'])

In [ ]:
df['Day'] = df['Last_Update'].dt.day
df['Month'] = df['Last_Update'].dt.month
df['Week'] = df['Last_Update'].dt.week
df['WeekDay'] = df['Last_Update'].dt.weekday

In [ ]:
# Layout Customization
displayed_cols = ['Confirmed','Deaths','Recovered']
def fig_plot():
    fig = plt.figure(constrained_layout=True, figsize=(45,18))
    fig.suptitle('bold figure suptitle', fontsize=14, fontweight='bold')

    grid = gridspec.GridSpec(ncols=4, nrows=2, figure=fig)

    ax1 = fig.add_subplot(grid[0, :2])
    ax1.set_title('Daily Reports',fontsize='large', fontweight='bold')
    df.groupby(['Last_Update']).sum()[displayed_cols].plot(ax=ax1,linewidth=7.0)

    
fig_plot()


In [ ]:
# Layout Customization
displayed_cols = ['Confirmed','Deaths','Recovered']
def fig_plot():
    fig = plt.figure(constrained_layout=True, figsize=(18,9))
    grid = gridspec.GridSpec(ncols=4, nrows=2, figure=fig)

    ax1 = fig.add_subplot(grid[0, :2])
    ax1.set_title('Weekly Reports', fontsize='large', fontweight='bold')
    weekdays = df.groupby('Week').nth(-1)['Last_Update']
    df[df['Last_Update'].isin(weekdays)].groupby('Last_Update')[displayed_cols].sum().plot(kind='density',ax=ax1,linewidth=7.0)

    
fig_plot()


**Observation**

*Daily Report shows that case count is increasing day by day which means that the calculation for today case count = today case count + previous days case count included. So we will take the last date as the updated case count.*

In [ ]:
# f,ax=plt.subplots(1,2,figsize=(28,8))
# df['Country'].value_counts().plot.pie(ax=ax[0],shadow=True)
# ax[0].set_title('Share of Countries')
# ax[0].set_ylabel('Count')
# sns.countplot('Country',data=df,ax=ax[1],order=df['Country'].value_counts().index)
# ax[1].set_title('Count of Country')
# plt.show()

df_countries = pd.DataFrame(df.country.value_counts().reset_index().values, columns=["country", "count"])
df_countries.head()

Count of Coronavirus cases detected in various countries

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Bar(
            x=df_countries['count'].tolist(),
            y=df_countries['country'].tolist(),
            orientation='h'))

fig.show()

**Observation:**

Mainland China accounts for the most of the cases i.e 1513 but significant number of cases were also observed for US*

In [ ]:
date = pd.DataFrame(df.Last_Update.value_counts().reset_index().values, columns=["Date", "Death"])
date.head()

Counts of deaths due to Coronavirus datewise 

In [ ]:
import plotly.express as px
# df = date
fig.update_layout(template='plotly_dark')
fig = px.scatter(date, x="Date", y="Death")

# If you print fig, you'll see that it's just a regular figure with data and layout
# print(fig)

fig.show()

**Observation**

*As we can clearly see with increase in date deaths due to COVID-19 are decreasing.*

In [ ]:
import plotly.express as px
fig.update_layout(template='plotly_dark')
# df = px.data.gapminder().query("continent == 'Europe' and year == 2007 and pop > 2.e6")
fig = px.bar(df, y='Recovered', x='ObservationDate', text='Confirmed')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

**Observation:**

*The study also noted that people who are infected begin to develop antibodies to the virus quickly, typically within six to 12 days. The rapid rise of antibodies may explain why about 80% of people infected with the virus do not develop severe disease and recover from it.*

In [ ]:
import plotly.express as px
import plotly.express as px
# df = px.data.gapminder().query("country=='Brazil'")
fig = px.line_3d(df, x="Confirmed", y="Deaths", z="Recovered",color = "country")
fig.show()

In [ ]:
 conf_df["Lat"].mean()
  

In [ ]:
conf_df["Long"].mean()  

In [ ]:
import folium
from folium.plugins import HeatMap
m=folium.Map([37,110],zoom_start=11)
HeatMap(conf_df[['Lat','Long']].dropna(),radius=18,gradient={0.2:'blue',0.4:'purple',0.6:'orange',1.0:'red'}).add_to(m)
display(m)

In [ ]:
# from mpl_toolkits.basemap import Basemap
# plt.figure(figsize=(8, 8))
# m = Basemap(projection='ortho', resolution=None, lat_0=30, lon_0=100)
# m.bluemarble(scale=0.5);

In [ ]:
 conf_df_ = conf_df[conf_df['province'].notna()]

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(conf_df_, lat="Lat", lon="Long", hover_name="province", hover_data=["country"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=400)
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
recent_date = df['Last_Update'].iloc[-1]
last_updated = df[df['Last_Update'].dt.date == recent_date]


In [ ]:
plt.figure(figsize=(15,6))
plt.title('Number of Province/State were affected in Each Country')
plt.xticks(rotation=90)
prv_lst = df.groupby(['country'])['province'].nunique().sort_values(ascending=False).head()
prv_lst.plot(kind='bar')
plt.tight_layout()

**Observation:**
*In US more than 175 Province were affected due to Covid-19.*

**Tweets data**

In [ ]:
df_tweets = pd.read_csv("/kaggle/input/covidtweets/nCoV_tweets.csv", index_col=0)
df_tweets.rename(columns={'txt': 'tweets',
                         'dt':'date'}, inplace=True)

import re
def tweet_parser(text, pattern_regex):
    
    for pr in pattern_regex:
        text = re.sub(pr, ' ', text)
        
    return text.strip()

pattern_regex = ['\n', '\t', ':', ',', ';', '\.', '"', "''", 
                 '@.*?\s+', 'RT.*?\s+', 'http.*?\s+', 'https.*?\s+']

df_tweets['tidy_tweets'] = df_tweets.apply(lambda r: tweet_parser(r['tweets'], pattern_regex), axis=1)
df_tweets['date'] = pd.to_datetime(df_tweets['date']).dt.date

df_tweets.head().style.background_gradient(cmap='RdGy')

Using TextBlob library to infer tweet sentiments, and later categorize them into Negative, Neutral and Positive.

In [ ]:
df_tweets['sentiment'] = df_tweets.apply(lambda r: TextBlob(r['tidy_tweets']).sentiment.polarity, axis=1)
df_tweets['sent_adj'] = np.where(df_tweets['sentiment']<0, 'Negative', np.where(df_tweets['sentiment']>0, 'Positive', 'Neutral'))
df_tweets['sent_adj'] = df_tweets['sent_adj'].astype('category')
sizes = df_tweets.groupby('sent_adj').size()

df_tweets.head().style.background_gradient(cmap='PuOr')

In [ ]:
fig = ff.create_distplot([df_tweets['sentiment']], group_labels = ['sentiment'], bin_size=[.05], colors=['indianred'])
fig.update_layout(
        title={'text': 'Sentiment Distribution',
               'y':0.95, 'x':0.5,
               'xanchor': 'center', 'yanchor': 'top'},
        showlegend=False)

fig.show()

In [ ]:

fig = go.Figure()
fig.add_trace(
    go.Bar(x=sizes.index,
           y=sizes.values,
           opacity=0.9,
           text = sizes.values,
           textposition='outside',
           marker={'color':'green'}
                   ))
fig.update_layout(
      title={'text': 'Sentiment Adjusted Histogram',
             'y':0.95, 'x':0.5,
            'xanchor': 'center', 'yanchor': 'top'},
       showlegend=False,
       xaxis_title_text='Sentiment',
        yaxis_title_text='Count',
    bargap=0.3)

fig.show()

In [ ]:
def render_wordcloud(df, sent='Positive'):
    
    color = {'Positive': 'Set2', 'Negative': 'RdGy', 'Neutral': 'Accent_r'}
    
    words = ' '.join([text for text in df.loc[df['sent_adj']==sent, 'tidy_tweets']])
    
    wordcloud = WordCloud(width=800, height=500, 
                          background_color='black',
                          max_font_size=100, 
                          relative_scaling=0.1, 
                          colormap=color[sent]).generate(words)

    plt.figure(figsize=(14, 10))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.title(sent + ' Wordcloud', fontsize=20)
    plt.axis('off')
    plt.show()
    

In [ ]:
for s in ['Positive', 'Negative', 'Neutral']:
    render_wordcloud(df_tweets, s)

**To prevent Coronavirus from spreading follow the below:**

* Avoid using towels/soaps/ clothes used by any other person.

* Keep cleaning your hands with a sanitizer especially after travelling in public transport like trains/ buses/ the metro etc.

* Keep yourself away from crowded areas.

* Always insist on wearing a mask especially use them for children.

* Drink only hot water

* Keep your home, kitchen and bathroom dry. Viruses thrive in musty environments.

* Have only fresh, hot, home cooked food.